# Pipeline to clean the keyword column 

In [68]:
import json
import pandas as pd
import itertools # to flatten lists of lists

## Load data

In [3]:
# Opening JSON file
f = open('../data/raw/CMS_2010_to_June_2022_ENGLISH.json')
  
# returns JSON object as a dictionary
data = json.load(f)

# convert to data frame
df = pd.DataFrame.from_dict(data)

In [5]:
df.columns

Index(['id', 'name', 'shortTitle', 'text', 'teaser', 'shortTeaser', 'kicker',
       'regions', 'keywords', 'keywordStrings', 'thematicFocusCategory',
       'navigations', 'categories', 'departments', 'firstPublicationDate',
       'lastModifiedDate', 'contentDate', 'relatedAutoTopics', 'contentLinks',
       'articles', 'isOpinion', 'geographicLocations', 'contentAssociations',
       'mainContentImageLink', 'images', 'externalLinks', 'topStory',
       'language'],
      dtype='object')

In [124]:
keywords = list(df.keywordStrings)

## Functions

In [147]:
# Used mainly for visualisation, get indices of keywords with a certain substring
def get_items_with_substring(lst_lst_keywords, substring):
    indices = [i for i, lst_kw in enumerate(lst_lst_keywords) if any(list(map(lambda x: substring in x, lst_kw)))]
    return indices

## Cleaning

### Cleaning 1: put everything in lowercase

In [148]:
keywords_lower = [list(map(str.casefold, x)) for x in keywords]

In [149]:
# for visualisation only (can remove later on)
print('before:', keywords[0])
print('after: ', keywords_lower[0])

before: ['DRC', 'M23', 'FDLR', 'Rwanda', 'Susan Rice', 'UN security council']
after:  ['drc', 'm23', 'fdlr', 'rwanda', 'susan rice', 'un security council']


### Cleaning 2: split keywords that haven't been split

In [150]:
# What we are looking for to split by
substring = ', '

# Split keywords: kw.split splits the keyword in a list of multiple keywords based on substring, itertools.chain flattens the list of lists
keywords_lower_split = [list(itertools.chain(*[kw.split(substring) for kw in lst_kw])) for lst_kw in keywords_lower]

# for visualisation only (can remove later on)
items_with_unsplit_keywords = get_items_with_substring(keywords_lower, substring)
print('before:', keywords_lower[items_with_unsplit_keywords[1]])
print('after: ', keywords_lower_split[items_with_unsplit_keywords[1]])

before: ['#speakup! barometer', 'freedom of speech, press freedom, freedom of expression', 'ghana']
after:  ['#speakup! barometer', 'freedom of speech', 'press freedom', 'freedom of expression', 'ghana']


### Cleaning 3: remove unwanted characters

In [155]:
# Remove '\u2002' and '.', replace '\xa0' with space
keywords_lower_split_clean = keywords_lower_split
keywords_lower_split_clean = [list(map(lambda x: x.replace('\u2002', ''), lst_kw)) for lst_kw in keywords_lower_split_clean]
keywords_lower_split_clean = [list(map(lambda x: x.replace('.', ''), lst_kw)) for lst_kw in keywords_lower_split_clean]
keywords_lower_split_clean = [list(map(lambda x: x.replace('\xa0', ' '), lst_kw)) for lst_kw in keywords_lower_split_clean]

In [156]:
# for visualisation only (can remove later on)
items_with_unwanted_characters = get_items_with_substring(keywords_lower, substring = '\u2002')
print('before:', keywords_lower_split[items_with_unwanted_characters[0]])
print('after: ', keywords_lower_split_clean[items_with_unwanted_characters[0]])

before: ['usa', 'trade pact', 'ethiopia', 'agoa', 'mali', '\u2002guinea']
after:  ['usa', 'trade pact', 'ethiopia', 'agoa', 'mali', 'guinea']


### Cleaning 4: other

In [158]:
items = [i for i,lst_kw in enumerate(keywords_lower) if any(list(map(lambda x: 'tropical tiger' in x, lst_kw)))]
print(keywords_lower_split_clean[items[0]])

['tropical tiger asian bush mosquito  mosquito', 'diseases']


In [160]:
items = [i for i,lst_kw in enumerate(keywords_lower) if any(list(map(lambda x: 'beijing olympics speedskating' in x, lst_kw)))]
print(keywords_lower_split_clean[items[0]])

['beijing olympics speedskating doping gold medal']


In [163]:
items = [i for i,lst_kw in enumerate(keywords_lower) if any(list(map(lambda x: 'allies of donald' in x, lst_kw)))]
print(keywords_lower_split_clean[items[0]])

['allies of donald trump are set to challenge the election result in a longshot that is almost certain to fail']
